In [3]:
import sys
sys.path.append('..')
from consts import MY_OPENAI_API_KEY as API_KEY
CACHE_DIR = '../cache/'
FIRST_PROMPT = "You are a helpful assistant designed to output JSON. The client provides you with text containing sales announcements, and you need to create JSON for these announcements to input into the database. The JSON should be a list of dictionaries, where each announcement is a separate dictionary. Each dictionary must include the fields: name, price, description (if available, otherwise null), place (information about the seller's location, the same for all products if provided, otherwise null), count (if multiple items are being sold at once, otherwise null), and others (a dictionary with other useful information about the product)."

In [4]:
import openai
import pickle as pkl
import pandas as pd
import typing as tp
from IPython.display import display
import json

In [5]:
openai_client = openai.OpenAI(api_key=API_KEY)

In [16]:
with open(CACHE_DIR + 'prompt_test_df.pkl', 'rb') as f:
  df = pkl.load(f)

In [180]:
class OpenAICache:
  def __init__(self, client):
    self.client = client
    self.storage = {}
  
  def request(self, prompt, text):
    if prompt not in self.storage:
      self.storage[prompt] = {}
    if text not in self.storage[prompt]:
      response = self.client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "system", "content": prompt},
          {"role": "user", "content": text}
        ]
      )
      self.storage[prompt][text] = response
    return self.storage[prompt][text]

class Prompt:
  def __init__(self, prompt:str, openai_cache:OpenAICache, fake_mode=False):
    self.prompt = prompt
    self.results = {}
    self.responses = []
    self.openai = openai_cache
    self.fake_mode = fake_mode
  
  def make_requests(self, df:pd.DataFrame, rewrite=False):
    for id in df.index:
      if id not in self.results or rewrite:
        response = self.openai.request(self.prompt, df.iloc[id]['text'])
        if not self.fake_mode:
          self.results[id] = json.loads(response.choices[0].message.content)
        else:
          self.results[id] = json.loads(response)

def make_flat_dict(obj):
  if isinstance(obj, list):
    obj = {str(i):obj[i] for i in range(len(obj))}
  if not isinstance(obj, dict):
    return obj
  new_obj = {}
  for key in obj.keys():
    val = make_flat_dict(obj[key])
    if isinstance(val, dict):
      for subkey in val.keys():
        new_obj[str(key) + '_' + subkey] = val[subkey]
    else:
      new_obj[str(key)] = val
  return new_obj

class PromptManager:
  def __init__(self, df, openai_cache):
    self.df = df
    self.openai = openai_cache
    self.prompts = {}
    self.baseline_name = None
    self.fake_mode = False

  def add(self, name, prompt:str):
    if name not in self.prompts:
      self.prompts[name] = Prompt(prompt, openai_cache=self.openai, fake_mode=self.fake_mode)

  def make_requests(self, name):
    self.prompts[name].make_requests(self.df)
  
  def compare(self, name1, name2):
    res1 = make_flat_dict(self.prompts[name1].results)
    res2 = make_flat_dict(self.prompts[name2].results)

    diff = []
    for key in set(res1.keys())|set(res2.keys()):
      val1 = res1[key] if key in res1 else '<no key>'
      val2 = res2[key] if key in res2 else '<no key>'
      if val1 != val2:
        col1 = name1 + (' (baseline)' if name1 == self.baseline_name else '')
        diff.append({'field':key, col1:val1, name2:val2})

    display(pd.DataFrame(diff))   
    return diff

  def compare_to_baseline(self, name):
    return self.compare(self.baseline_name, name)

  def make_all(self, name, prompt):
    self.add(name, prompt)
    self.make_requests(name)
    if self.baseline_name is not None:
      self.compare_to_baseline(name)
    else:
      self.baseline_name = name

  def set_baseline(self, name):
    self.baseline_name = name

  def print_prompt(self, name):
    print(self.prompts[name].prompt)

In [181]:
# tests
REQ = 'Всем доброго дня! \n\nПродаю туфли, метро Новокосино. \n\n1. Бежевые лакированные туфли, размер 35. Абсолютно новые. — 1000р. \n2. Синие туфли, размер 36. Абсолютно новые. — 1000р. \n3. Чёрные лакированные туфли из натуральной кожи марки Elmonte. Размер 36. Носились пару раз, в отличном состоянии. Стоят новые набойки и профилактика. — 1000р. \n4. Чёрные туфли под замшу. Размер 36. Носились недолго и аккуратно, в хорошем состоянии. — 500р.'
FAKE_PROMPT = 'lol'
class FakeCache(OpenAICache):
  def __init__(self):
    self.storage = {
      FIRST_PROMPT: {
        REQ: """[
          {
              "name": "Бежевые лакированные туфли",
              "price": 1000,
              "description": "Абсолютно новые",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 35}
          },
          {
              "name": "Синие туфли",
              "price": 1000,
              "description": "Абсолютно новые",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 36}
          },
          {
              "name": "Чёрные лакированные туфли из натуральной кожи марки Elmonte",
              "price": 1000,
              "description": "Носились пару раз, в отличном состоянии. Стоят новые набойки и профилактика.",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 36}
          },
          {
              "name": "Чёрные туфли под замшу",
              "price": 500,
              "description": "Носились недолго и аккуратно, в хорошем состоянии.",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 36}
          }
        ]"""
      },
      FAKE_PROMPT: {
        REQ: """[
          {
              "name": "Бежевые",
              "price": 1000,
              "description": "Абсолютно новые",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 35}
          },
          {
              "name": "Синие туфли",
              "price": 1000,
              "description": "Абсолютно новые",
              "place": "метро",
              "count": null,
              "others": {"Размер": 36}
          },
          {
              "name": "Чёрные лакированные туфли из натуральной кожи марки Elmonte",
              "price": 1000,
              "description": "Носились пару раз, в отличном состоянии. Стоят новые набойки и профилактика.",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 36}
          },
          {
              "name": "Чёрные туфли под замшу",
              "price": 400,
              "description": "Носились недолго и аккуратно, в хорошем состоянии.",
              "place": "метро Новокосино",
              "count": null,
              "others": {"Размер": 36}
          }
        ]"""
      }
    }

In [182]:
fc = FakeCache()
fpm = PromptManager(pd.DataFrame([{'text':REQ}]), fc)
fpm.fake_mode = True

In [183]:
fpm.make_all('req1', FIRST_PROMPT)

In [173]:
fpm.make_all('req2', FAKE_PROMPT)

,field,req1 (baseline),req2
0,0_3_price,500,400
1,0_0_name,Бежевые лакированные туфли,Бежевые
2,0_1_place,метро Новокосино,метро


In [184]:
import sys
sys.path.append('..')
from consts import LOL